In [1]:
# 例子: 分类猫和狗的图片
# 使用一个还不错的相机(12M像素) RGB的图片有36M像素
# 使用100大小的单隐藏层MLP, 模型有3.6B元素, 远多于世界上的所有猫和狗的总和

# 回顾 单隐藏层MLP
# 输入有36M个元素, 有100个神经元, 那么就有100*36M个weight

# 找Waldo, 两个原则:
# 1. 平移不变性(像素不改变) 2. 局部性(只用看一块,局部信息）

# 重新思考全连接层

In [2]:
# 傅立叶变换的例子 integral (-inf to inf) f(x) g(t-x)

# f(t)表示进食, g(t) 表示消化剩余物的比例
# 假设 12点吃入米饭, 想得到14点胃里米饭的比例就可以用公式
# f(12) * g(14-12)

# 假如说x时刻吃进的米饭, 想知道在t时刻的胃有多少食物, 就经历了(t-x)的时间
# 我们就可以放到g函数中, g的横坐标就是t-x小时之后, 随着t增大, 就表明剩下的食物比例越低, 类似一个(e^-x)的图
# 所以 f(x)*g(t-x) 的意思就是在x时间吃下了某个食物, 按照比例消化, 到t时刻还剩下多少(其实x就是个定量)
# 那么如果我们想知道从前面的每一个点到t时刻还剩下多少食物, 也就是吃下所有食物, 到t时刻胃里剩的东西
# 就是 integral (t to 0) f(x)g(t-x)dx

# 想知道是否是卷积, 还可以从f(x)中的x加上g(t-x)中的t-x是否会消除掉一个variable, 剩下另外一个来得知
# 这里的f(x)和g(t-x)在图形上对应的是什么呢？我们可以理解f(t)函数和g(t)函数是一一对应相连的关系
# 具体可以说, 在f(t)中的t越大, 就可以连接g(t)中t越小的地方, 表示为我刚刚吃下东西, 还没怎么消化 
# 反之, f(t)中的t越小, 就可以连接g(t)中t越大的地方, 表示为我吃下东西很久了, 基本消化完了
# 所以做‘integral’就是得到之前吃的所有东西到现在还剩的食物

# 卷积的物理意义就是: 有一个系统, 输入是不稳定的, 输出是稳定的, 就可以中卷机来求系统的存量
# 如果把g(t)函数图翻转一下, 就可以直观得到这个消化例子的一一对应

In [3]:
# 那么图像的卷积操作是什么呢? 
# 使用卷积核在对应图像上做相乘, 求和, 最后将得到的值保存在一个格子上, 也就是一个新的像素点
# 将整个图像按这样做卷积操作后, 就可以得到一个新的图像
# 因为这样处理后少了一圈, 可以用padding在这些空的格子填0, 就可以得到一个同样大小的图片

# 怎样理解是一个卷积呢?
# 图像是一个不稳定的输入, 每次都不一样, 而用来输出的是一个卷积核, 是稳定的
# 但这样很难理解, 换个例子

# 假如还是用f(t)和g(t)函数, 
# f(t) 在t时刻发生了飓风, 发生飓风的原因是x时刻有很多蝴蝶扇动了翅膀
# g(t) 则表示蝴蝶扇动翅膀对发生飓风的影响, g(t)是随着t增而降低
# 而这里的卷积则是: 在某一时刻发生了一件事, 而这件事的产生会受到之前发生事情的影响
# 在这里就是x时刻发生了一件事, 对t时刻的事情有一定的影响, 怎么影响呢, 还是会考虑x到t时刻经历的时间发生的变化, 就是g函数
# 回到图像的卷积操作, 是不是就是图像上很多个像素点对一个像素点是怎么影响的

# f(x,y)*g(m,n) = sigma f(x,y) dot g(m-x, n-y)
# 现在只考虑举例(x,y) 相邻1个像素的影响
  
# f(x-1, y+1)   f(x, y+1)   f(x+1, y+1)                g(-1,1)    g(0,1)   g(1,1)
# f(x-1, y)     f(x, y)     f(x+1, y)                  g(-1,0)    g(0,0)   g(1,0)
# f(x-1, y-1)   f(x, y-1)   f(x+1, y-1)                g(-1,-1)   g(0,-1)  g(1,-1)
# 假如想知道 f(x-1, y-1) 对 f(x,y)有什么影响, 也就是g(x-(x-1), y-(y-1)), 找g(1,1)
# 将g函数旋转180度, 就可以表示为‘卷积核’

# 卷积神经我网络是做什么的？
# 将图像的局部特征提取出来, 交给网络
# 其实卷积操作就是过滤某些特征, 得到某些特征
# 可以理解为一种试探, 当不想得到某些特征, 可以将这个地方设置成0

In [4]:
# 回到重新考虑全连接层
# 1. 将输入和输出变换为矩阵
# 2. 将权重变成一个4-D的张量
#     h_i,j = sigma(k,l) w_{i,j,k,l} x_{k,l} = sigma(a,b) v_{i,j,a,b} x_{i+a, j+b}
#     这里的x是我们的输入, 变为矩阵之后有k,l两个选项
#     这里的w就是全连接层的权重, 这是一个四维的矩阵
# 3. 在w_{i,j,k,l}中; i,j代表输出矩阵中的位置, k,l代表输入矩阵中的位置; 
#    每个权重描述输入矩阵中位置为(k,l)点如何影响输出矩阵中位置为(i,j)的点
#    # 举例说明: 假设输入图像为4x4, 输出图像为2x2, 输出图像中每一个像素, 
     # 需要记录输入图中(1,1), (1,2), ..., (2,1), ..., (4,4)这些所有的点对输出图(1,1)的影响
     # 同理也需要记录这些所有点对输出图中(1,2), ...., (2,2)的影响
     # 那么这时候对于每一组点就有4个参数：输入图的横坐标、纵坐标，输出图的横坐标、纵坐标, 也就是四维
# 4. 写成这个形式之后, 我们可以写一个索引, 将w的元素重新排列, 写到v上面
#    V是W的重新索引, v_{i,j,a,b} = w_{i,j,i+a,j+b}

In [ ]:
# 平移不变性
# x的平移会导致h的平移, h_{i,j} = sigma(a,b) v_{i,j,a,b} x_{i+a, j+b}
# 意思是说不管输入i,j怎么变换, v应该是不变的
# 解决方案: 加一个限制, 让v_{i,j,a,b} = v_{a,b}
# h_{i,j} = v_{a,b} x_{i+a,j+b}
# 其实就是低维度平移不变性对权重做了限制, 然后把i,j这个维度干掉了

# 局部性
# h_{i,j} = sigma(a,b) v_{a,b}x_{i+a,j+b}
# 当评估h_{i,j}, 我们不应该用远离x_{i,j}的参数
# 解决方案: 当|a|, |b| > delta时, 使得v_{a,b} = 0
# h_{i,j} = sigma delta (a = -delta) sigma delta (b= -delta) v_{a,b}x_{i+a,j+b}
# 就是说宽度和长度的感受野在-delta到delta

In [5]:
# 总结
# 对全连接层使用平移不变性和局部性得到卷积层
# h_{i,j} = sigma(a,b) v_{i,j,a,b} x_{i+a, j+b} = sigma delta (a = -delta) sigma delta (b= -delta) v_{a,b}x_{i+a,j+b}

In [6]:
# 卷积层
# 二维交叉相关
# Input: 3x3的矩阵 *（卷积) Kernel: 2x2的矩阵 = Output
# 通过dot product, 将 3x3矩阵变成一个2x2的Output, 每个Output的grid都是点积(dot)的和

# 二维卷积层
# 输入X: n_h x n_w
# Kernel: k_h x k_w
# 偏差b in R
# 输出Y: (n_h - k_h +1) x (n_w - k_w +1) stride默认为1
# Y = X * W +b, 这里W和b是可学习的参数

In [7]:
# 交叉相关 vs. 卷积
# 二维交叉相关
# y_{i,j} = sigma(a=1 to h)sigma(b=1 to w) W_{a,b} X_{i+a,j+b}
# 二维卷积
# y_{i,j} = sigma(a=1 to h)sigma(b=1 to w) W_{-a,-b} X_{i+a,j+b}
# 区别只有 W_{a,b}的负号, 由于对称性, 在实际使用中没有区别

# 一维和三维交叉相关性
# 一维
# y_i = sigma(a=1 to h) W_{a}X_{i+a}
# 文本, 语言, 时序序列

# 三维
# y_{i,j,k} = sigma(a=1 to h) sigma(b=1 to w) sigma(c=1 to d) W_{a,b,c} x_{i+a, j+b, k+c} 
# 视频, 医学图像, 气象地图

# 总结
# 卷积层将输入和核矩阵进行交叉相关, 加上偏移后得到输出
# 核矩阵和偏移是可学习的参数
# 核矩阵的大小是超参数

In [9]:
# 图像卷积

# 互相关运算
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K): # X是输入, K是核矩阵
    """计算二维互相关运算"""
    h, w = K.shape # Kernel的height, width
    Y = torch.zeros((X.shape[0] - h +1, X.shape[1] - w +1)) # Y是Output的大小, 根据前面公式
    for i in range(Y.shape[0]):    
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i + h, j:j + w] * K).sum() # Nested loop进行更新, 每次取出X中Kernel大小的区域, 与K做dot, 并求和
    return Y

# 验证上述二维互相关运算的输出
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [11]:
# 实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size)) # weight和bias是可学习的参数
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias 
    
# 卷积层的一个简单应用: 检测图像中不同颜色的边缘
X = torch.ones((6, 8))
X[:, 2:6] = 0
X
# 这里从1到0可以理解为是一个vertical的边缘

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [15]:
K = torch.tensor([[1.0, -1.0]]) # 一个1x2的核
# 假如两个元素没变 -> 输出是0; 假设变了 -> 输出是-1

# 输出Y中的1代表从白色到黑色的边缘, -1/1代表黑色到白色的边缘
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [14]:
# 卷积核只能检测垂直的边缘, transpose之后就检测不到东西了
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [16]:
# 学习由X生成Y的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False)

X = X.reshape((1, 1, 6, 8)) 
Y = Y.reshape((1, 1, 6, 7))

# 第一个参数1是‘输入通道数’, 意味每个样本只有一个通道, 例如灰度图像
# 第二个参数2是‘输出通道数‘, 表明卷积层会产生一个单通道的输出
# kernel_size = (1,2) 是卷积核的大小

# X, Y中的第一个1是batch_size, 第二个1是通道数, 与conv2d的第一个1相对应; 6,7 和 6,8是每个样本的空间维数
# 在这, X的宽度为8, 经过宽为2的卷积操作后, 输出为Y的宽度8-2+1 = 7

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2 # 平方差
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad # 这里 3e-2是学习率
    if (i + 1)%2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')
        

batch 2, loss 15.998
batch 4, loss 5.205
batch 6, loss 1.906
batch 8, loss 0.743
batch 10, loss 0.298


In [17]:
# 所学的卷积核的权重张量
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9318, -1.0437]])